In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re

from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 경고표시 지우기
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
filepath = "./data/model/model_lstm_220110.h5"

model = load_model(filepath, custom_objects=None, compile=True)

In [3]:
dir(model)

['_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_variable_with_custom_getter',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_base_model_initialized',
 '_build_graph_network_for_inferred_shape',
 '_build_input_shape',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_arg_defaults',
 '_call_fn_arg_positions',
 '_call_fn_args',
 '_call_full_argspec',
 '_callable_losses',
 '_cast_single_input',
 '_check_call_args',
 '_checkpoint_dependencies',
 '_clear_los

In [4]:
def clean_str(string):    
    string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"\'{2,}", "\'", string)
    string = re.sub(r"\'", "", string)

    return string.lower()

df = pd.read_csv("./data/review_data/total_review.csv", encoding="utf-8-sig", index_col=0)
review_df = df.dropna(axis=0)

# 컬럼 재배치
# review_df = review_df[["review","score"]]

# clean_str 함수를 통해서 X데이터를 정제해주어야함.
# train_test_X = [clean_str(sentence) for sentence in review_df["review"]]
# # 문장을 띄어쓰기 단위로 단어 분리
# sentences = [sentence.split(' ') for sentence in train_test_X]
# for i in range(5):
#     print(sentences[i])

# # 문장의 길이를 그래프화
# sentence_len = [len(sentence) for sentence in sentences]
# sentence_len.sort()
# plt.plot(sentence_len)
# plt.show()

# # 총 3956974행의 데이터 중 30문장이내의 데이터는 3401248으로 85.9%에 육박.
# # 따라서, padding의 기준을 30로 잡겠음.
# print(sum([int(l<=30) for l in sentence_len]))
    
# sentence_new = []
# for sentence in sentences:
#     sentence_new.append([word[:5] for word in sentence][:30])

# sentences = sentence_new

# for i in range(5):
#     print(sentences[i])

# tokenizer = Tokenizer(num_words=3000)
# tokenizer.fit_on_texts(sentences)

In [5]:
mango = pd.read_excel("./data/망고플레이트 리뷰 종합.xlsx")

mango

,Unnamed: 0,이름,평점,주소,주력 메뉴,가격대,망고플레이트 주소,리뷰내용
0,3869.0,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 가로수길에 있는 오랜 참치맛집 e-청우참치위치를 옮기다!!원래...
1,3865.0,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,"\n 가성비 너무 좋고, 정말 양질의 참치를 양껏 즐길 수 있어서 ..."
2,3867.0,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 가성비 좋은 한국식 참치횟집. 선도가 꽤 좋다. 카운터석도 있...
3,3868.0,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 그저 박수만. 참치땡길때 여기만한 곳이 없다.\n
4,3863.0,청우참치,\n4.5\n,서울특별시 강남구 강남대로 632,\n회 / 스시\n,4만원 이상,https://www.mangoplate.com//restaurants/YyRGx0...,\n 맛있다.. 느끼해서 많이 못먹는게 함정.\n
...,...,...,...,...,...,...,...,...
36203,NaN,키미노돼지국밥,-,서울특별시 중랑구 중랑천로10길 92 1F,\n탕 / 찌개 / 전골\n,만원 미만,https://www.mangoplate.com//restaurants/OGDe64...,\n 뜨끈한 돼지국밥 생각 날 때 자주 찾는 곳.멀리서도 먹으러 찾...
36204,NaN,조개나루터,-,서울특별시 중랑구 중랑천로12길 59,\n해산물 요리\n,NaN,https://www.mangoplate.com//restaurants/MgyUt1...,\n 11월말에 처음 가보고나서 12~1월두달동안 벌써 8번이나 다...
36205,NaN,조개나루터,-,서울특별시 중랑구 중랑천로12길 59,\n해산물 요리\n,NaN,https://www.mangoplate.com//restaurants/MgyUt1...,\n 가리비크기 미친거아닌가요.......내일 또갈래요..........
36206,NaN,제주돈선생,-,서울특별시 중랑구 중랑천로12길 61,\n고기 요리\n,만원-2만원,https://www.mangoplate.com//restaurants/JtAAHT...,"\n 굳이 없는 식당까지 등록해가며 쓰지 않는데, 고기도 맛있고 사..."


In [6]:
mango_sentence = []
for review in mango.iloc[:20,-1]:
    mango_sentence.append(review)
    
mango_sentence

mango_score = []
for score in mango.iloc[:20,2]:
    mango_score.append(score)
    
mango_score

['\n4.5\n',
 '\n4.5\n',
 '\n4.5\n',
 '\n4.5\n',
 '\n4.5\n',
 '\n4.5\n',
 '\n4.5\n',
 '\n4.5\n',
 '\n4.5\n',
 '\n4.5\n',
 '\n4.5\n',
 '\n4.5\n',
 '\n4.5\n',
 '\n4.5\n',
 '\n4.5\n',
 '\n4.3\n',
 '\n4.3\n',
 '\n4.3\n',
 '\n4.3\n',
 '\n4.3\n']

In [8]:
cnt = 0

# 망고플레이트 리뷰 테스트
for review in mango_sentence:
    test_sentence = review
    test_sentence = test_sentence.split(' ')
    test_sentences = []
    now_sentence = []
    for word in test_sentence:
        now_sentence.append(word)
        test_sentences.append(now_sentence[:])
        
    tokenizer = Tokenizer(num_words=3000)
    tokenizer.fit_on_texts(test_sentences)
        
    test_X_1 = tokenizer.texts_to_sequences(test_sentences)
    test_X_1 = pad_sequences(test_X_1, padding='post', maxlen=30)
    prediction = model.predict(test_X_1)
    for idx, sentence in enumerate(test_sentences):
#         print(sentence)
#         print((prediction[idx]).round(3))
        review_score = np.argmax((prediction[idx]).round(3))
    # print(review)
    print(review_score)
    print(mango_score[cnt])
    print("-"*70)
    cnt+=1

# tmp_score_list = []
# cnt = 1

# for data in review_score:
#     if not tmp_score_list:
#         tmp_score_list.append(data)
#     else:
#         tmp_score_list.append(data)
#         if tmp_score_list[0] > tmp_score_list[1]:
#             del tmp_score_list[1]
#         else:
#             del tmp_score_list[0]
    
# print(tmp_score_list)
    
# print(tmp_score_list)
# 띄어쓰기 기준으로 끊는 것이니 

3

4.5

----------------------------------------------------------------------
4

4.5

----------------------------------------------------------------------
4

4.5

----------------------------------------------------------------------
4

4.5

----------------------------------------------------------------------
4

4.5

----------------------------------------------------------------------
5

4.5

----------------------------------------------------------------------
4

4.5

----------------------------------------------------------------------
4

4.5

----------------------------------------------------------------------
5

4.5

----------------------------------------------------------------------
4

4.5

----------------------------------------------------------------------
5

4.5

----------------------------------------------------------------------
5

4.5

----------------------------------------------------------------------
4

4.5

--------------------------------------------

In [9]:
# 망고 플레이트에서 가게 랜덤으로 뽑아서 점수 예측.

import random

cnt = 0

mango_store_list = []

for store in mango["이름"].unique():
    mango_store_list.append(store)

choiceList = random.choice(mango_store_list)

data = mango[mango["이름"]==choiceList]
print("뽑은 가게는 {}입니다.".format(str(data["이름"].unique())))

# 망고플레이트 리뷰 테스트
for review in data.iloc[:,-1]:
    test_sentence = review
    test_sentence = test_sentence.split(' ')
    test_sentences = []
    now_sentence = []
    for word in test_sentence:
        now_sentence.append(word)
        test_sentences.append(now_sentence[:])

    test_X_1 = tokenizer.texts_to_sequences(test_sentences)
    test_X_1 = pad_sequences(test_X_1, padding='post', maxlen=30)
    prediction = model.predict(test_X_1)
    for idx, sentence in enumerate(test_sentences):
#         print(sentence)
#         print((prediction[idx]).round(3))
        review_score = np.argmax((prediction[idx]).round(3))
    # print(review)
    print("리뷰\n", review)
    print("실제 점수 : ", data.iloc[cnt,2])
    print("예측한 점수 : ", review_score)
#     print(mango_score[cnt])
    print("-"*70)
    cnt+=1

뽑은 가게는 ['뽀르누서울']입니다.
리뷰
 
           서울의 놀리타?! 와 어쩜 이런 곳에!!! 이 동네를 살지 않고서야 이 곳의 존재를 알기도, 먹으러 가기도 쉽지 않아요. 하... 이렇게 한숨 나오는 이유는 찾기 애매하지만 맘에 들어서 또 와야하거든요. 진짜 작은 곳인데도 사람들이 잘 찾아오네요, 안에 테이블이 한 세개 있나..? 샌드위치나 수프등 음식이 빨리 솔드아웃됩니다.  “까놀리”☆ - 인생 첫 까놀리였는데, 바로 맛있음을 느꼈어요. 페이스트리껍질은 가볍고 매우 바삭!해요. 크림은 피스타치오맛인데 매우 맛있네요, 고소하고 달콤하고 라이트해요. 생각보다 커서 좋았는데 그래도 하나로는 부족!! “포카치아 샐러드 보울” - 먹기 전엔 약간 가격이 비싼듯했지만 포카치아 맛있으니 가격 생각은 바로 사라졌어요. 진짜 이탈리아는 디저트까지 맛없는 게 없네!!다음에 또 와서 까놀리도 여러개 사고 샌디치도 티라미스도 바치디다마도 먹을 거에요. 다시 영업을 시작하실 날짜를 기다리며...
        
실제 점수 :  
4.4

예측한 점수 :  4
----------------------------------------------------------------------
리뷰
 
          "한남동에서 느끼는 이탈리아의 맛."이탈리안 브런치 맛집 "뽀르누서울"에 가보았어요.미케타 샌드위치(+ 프로슈토 선택), 버섯 수프, 카놀리. 세 가지 주문해 먹어보았습니다.미케타 샌드위치는 바작한 빵 안에 두 가지 치즈, 바질 페스토, 루꼴라, 썬드라이 토마토가 들어가는 샌드위치예요.프로슈토, 치즈의 짭짤 꼬릿함과 썬드라이 토마토의 쫄깃함이 인상적이었어요. 진한 버섯 수프에는 크루통이 진짜 듬뿍 들어가 있었어요.오래 끓인 들깨국 느낌도 좀 나는 게 완전 제 스타일이었습니다. 다음에 가도 또 주문해 먹을 것 같아요.이탈리아 수제 디저트라는 카놀리는 바삭한 쉘 안에 부드럽고 은은한 피스타치오 리코타 크림이 

In [43]:
# 망고플레이트 리뷰 테스트
store = input("가게 이름을 입력하시오 : ")

data = mango[mango["이름"]==store]
if data.empty is True:
    print("데이터가 없습니다.")

else:
    score_sum = 0
    cnt=0

    for review in data.iloc[:,-1]:
        test_sentence = review
        test_sentence = test_sentence.split(' ')
        test_sentences = []
        now_sentence = []
        for word in test_sentence:
            now_sentence.append(word)
            test_sentences.append(now_sentence[:])

        test_X_1 = tokenizer.texts_to_sequences(test_sentences)
        test_X_1 = pad_sequences(test_X_1, padding='post', maxlen=30)
        prediction = model.predict(test_X_1)
        for idx, sentence in enumerate(test_sentences):
            review_score = np.argmax((prediction[idx]).round(3))
        print("리뷰\n", review)
        print("실제 점수 : ", data.iloc[cnt,2])
        print("예측한 점수 : ", review_score)
        print("-"*70)
        score_sum += review_score
        cnt+=1
    print("예측한 평균 평점 : {}".format((score_sum/cnt+1).round(2)))

가게 이름을 입력하시오 : 청우참치
리뷰
 
          가로수길에 있는 오랜 참치맛집 e-청우참치위치를 옮기다!!원래 있던 자리의 건물이 매각되어 바로 인근으로 옮기고 살짝 리모델링(?)까지는 아니지만 옮기고 아주 살짝 넒어진 느낌이지만 그래도 바테이블에 앉으면 엄청 다닥다닥 붙어 앉아야 하지만 정감가는 매력이 있는곳.이곳에서는 참치는 부위별로 다양한 맛을 즐길 수 있는데~ 원래 빨간부위를 좋아했는데 청우참치에서 다른 부위의 맛도 즐기게 됨 마블링있는 부위는 마치 소고기를 먹는것처럼 살짝 기름지면서 고소하고 부드러움 ㅠㅠ 더 많이 먹었어야 했는데 내 위가 왜 허락칠 않는거니ㅠㅠ 쓸데없이 감자랑 콘버터까지 맛있음... 예약을 잘 받지않아 대기해야하고 주말에도 예전에는 영업 안했는데 가시려면 확인하고 가야해요~^^
        
실제 점수 :  
4.5

예측한 점수 :  5
----------------------------------------------------------------------
리뷰
 
          가성비 너무 좋고, 정말 양질의 참치를 양껏 즐길 수 있어서 더 좋다. 참치하면 청우참치 말고 다른 곳은 생각이 안난다...
        
실제 점수 :  
4.5

예측한 점수 :  4
----------------------------------------------------------------------
리뷰
 
          가성비 좋은 한국식 참치횟집. 선도가 꽤 좋다. 카운터석도 있고 바닥에 양반다리 하고 앉아먹기도 하는 아저씨스타일집. 단촛물밥 달라고 해서 초밥으로 싸먹으면 맛있다. 분위기나 멋 포기하고 오로지 참치만 생각한다면 추천
        
실제 점수 :  
4.5

예측한 점수 :  4
----------------------------------------------------------------------
리뷰
 
          그저 박수만. 참치땡길때 여기만한 곳이 없다.
        


In [23]:
##### 정리


##### import Library #####
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
import random

from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 경고표시 지우기
import warnings
warnings.filterwarnings(action="ignore")
##### import Library #####

##### function #####
def clean_str(string):    
    string = re.sub(r"[^가-힣A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"\'{2,}", "\'", string)
    string = re.sub(r"\'", "", string)

    return string.lower()

def model_load(file_path):
    print("모델 불러오는 중...")
    # model load
    filepath = file_path
    model = load_model(filepath, custom_objects=None, compile=True)
    
    return model

def data_load(dataframe):
    print("데이터 불러오는 중...")
    df = pd.read_csv(dataframe, encoding="utf-8-sig", index_col=0)
    review_df = df.dropna(axis=0)
    
    return review_df

def model_random_test(mango):
    print("모델 테스트 중...")
    # 망고 플레이트에서 가게 랜덤으로 뽑아서 점수 예측.
    cnt = 0
    score_sum = 0
    mango_store_list = []
    try:
        for store in mango["이름"].unique():
            mango_store_list.append(store)

        choiceList = random.choice(mango_store_list)

        data = mango[mango["이름"]==choiceList]
        print("뽑은 가게는 {}입니다.".format(str(data["이름"].unique())))

        score = 0
    except:
        print("망고플레이트 Database안에 없는 가게입니다.")

    # 망고플레이트 리뷰 테스트
    for review in data.iloc[:,-1]:
        test_sentence = review
        test_sentence = test_sentence.split(' ')
        test_sentences = []
        now_sentence = []
        for word in test_sentence:
            now_sentence.append(word)
            test_sentences.append(now_sentence[:])
            
        tokenizer = Tokenizer(num_words=3000)

        test_X_1 = tokenizer.texts_to_sequences(test_sentences)
        test_X_1 = pad_sequences(test_X_1, padding='post', maxlen=30)
        prediction = model.predict(test_X_1)
        for idx, sentence in enumerate(test_sentences):
            review_score = np.argmax((prediction[idx]).round(3))
            
        print("리뷰\n", review)
        print("실제 점수 : ", data.iloc[cnt,2])
        print("예측한 점수 : ", review_score)
        print("-"*70)
        
        score_sum += review_score
        cnt+=1
    print("예측한 평균 평점 : {}".format(((score_sum/cnt).round(2))))

def model_test(mango):
    print("모델 테스트 중...")
    print("리뷰 모델 테스트.")
    store = input("가게 이름을 입력하시오 : ")

    data = mango[mango["이름"]==store]
    if data.empty is True:
        print("Database에 해당 가게가 없습니다.")

    else:    
        score_sum = 0
        cnt=0

        for review in data.iloc[:,-1]:
            test_sentence = review
            test_sentence = test_sentence.split(' ')
            test_sentences = []
            now_sentence = []
            for word in test_sentence:
                now_sentence.append(word)
                test_sentences.append(now_sentence[:])

            tokenizer = Tokenizer(num_words=3000)
            
            test_X_1 = tokenizer.texts_to_sequences(test_sentences)
            test_X_1 = pad_sequences(test_X_1, padding='post', maxlen=30)
            prediction = model.predict(test_X_1)
            for idx, sentence in enumerate(test_sentences):
                review_score = np.argmax((prediction[idx]).round(3))
            print("리뷰\n", review)
            print("실제 점수 : ", data.iloc[cnt,2])
            print("예측한 점수 : ", review_score)
            print("-"*70)
            score_sum = score_sum + review_score
            cnt+=1
        print("예측한 평균 평점 : {}".format((score_sum/cnt).round(2)))
        
##### function #####

##### Main #####
if __name__  == "__main__":
    
    path = "./data/model/model_lstm_220110.h5"
    model = model_load(file_path = path)
    
    data = "./data/review_data/total_review.csv"
    df = data_load(data)
    
    # 망고플레이트 리뷰 데이터
    mango = pd.read_excel("./data/망고플레이트 리뷰 종합.xlsx")
    
    opinion = input("어느 기능을 사용하시겠습니까(1:작동테스트 / 2:가게입력) : ")
    if opinion == "1":
        model_random_test(mango=mango)
    elif opinion == "2":
        model_test(mango=mango)
    else:
        print("번호를 잘못 입력하셨습니다. 1번과 2번중 골라주세요.")

모델 불러오는 중...
데이터 불러오는 중...
어느 기능을 사용하시겠습니까(1:작동테스트 / 2:가게입력) : 1
모델 테스트 중...
뽑은 가게는 ['차고버거']입니다.
리뷰
 
           차고버거 (화-토 11:30-21:30 일 11:30-21:00평일 브레이크타임 15:00-17:00) 서울 송파구 오금로11길 61-11 1층 101호✔️ 차고버거 클래식, 치즈프라이꽤나 찾기 어려운 자리에 있는 차고버거.이 자리에 아 가게가? 싶을 수도 있는 위치지만 그래도 건재하게 잘 있는 가게이다. 몇 년동안 먹고 있는데 늘 변함없는 맛을 선사해줘서 먹으면 실망이 없다.- 차고버거 클래식은 클래식인 이름에 맞게 고기 패티와 다른 야채들로 만들어졌다. 적양파가 생으로 나오기 때문에 양파 맛이 강하다. - 치즈프라이가 특이하게 나초와 같이 나온다. 프라이가 그냥 두툼한 맛이라면 나초가 바삭해서 둘을 같이 먹으면 맛있다. 치즈프라이 위에는 치즈소스를 올리는데 비주얼이 딱 군침 도는 맛.늘 만족하면서 먹는 가게다. 이 가게의 장점은 두툼한 고기패티와 프라이 소스! 애견 동반도 가능한 가게이기 때문에 견주분들도 마음껏 이용 가능한 곳!⏰ 주말 점심 방문 기준 대기 없음.
        
실제 점수 :  
4.2

예측한 점수 :  5
----------------------------------------------------------------------
리뷰
 
          먹다가 급히 올리는 바람에 사진이 개떡같이 나왔지만정말 맛있다 배 안에 여유가 될것 같다면 프라이도 꼭 먹어보길 바란다.이곳의 유일한 단점은 길건너의 모텔인듯 하다. 
        
실제 점수 :  
4.2

예측한 점수 :  5
----------------------------------------------------------------------
리뷰
 
          무지막지한 감튀 사이즈. 진한 맛의 맛있는 수제버거. 집에 가려면 뒤뚱뒤뚱 배를 부여잡고 열심히 걸어